In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
# PATH
DATA_PATH  = '../DATA'
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TRAIN_CSV  = os.path.join(DATA_PATH, 'train.csv')
SAMPLE_PATH = os.path.join(DATA_PATH, 'sample_submission.csv')
LABEL_PATH = os.path.join(DATA_PATH, 'label.csv')

In [17]:
df_train = pd.read_csv(TRAIN_CSV)
df_train.head()

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050


In [6]:
from pykrx import stock

df_list = []
for item in tqdm(df_train['종목코드'].unique()):
    df = stock.get_market_ohlcv_by_date("2023-05-31", "2023-06-30", item[1:])
    df['종목코드'] = item
    df_list.append(df)
final_df = pd.concat(df_list)
final_df.to_csv(LABEL_PATH, index=False)


100%|██████████| 2000/2000 [08:22<00:00,  3.98it/s]


In [21]:
final_df

,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드
날짜,,,,,,,,
2023-05-31,2995,3445,2935,3020,40162146,128834191830,0.17,A060310
2023-06-01,3000,3070,2850,2900,5019544,14708943755,-3.97,A060310
2023-06-02,2905,2950,2825,2875,2358195,6798794605,-0.86,A060310
2023-06-05,2850,3220,2785,2935,18938294,57564475670,2.09,A060310
2023-06-07,2890,3060,2855,2890,4640916,13739139530,-1.53,A060310
...,...,...,...,...,...,...,...,...
2023-06-26,7900,7900,7390,7490,113422,850147020,-1.45,A238490
2023-06-27,7500,7550,7360,7420,56069,415886550,-0.93,A238490
2023-06-28,7420,7560,7410,7480,53417,399921010,0.81,A238490


In [26]:
def get_target(df):
    # change columns name
    df = df_train.rename(columns={
                                    '일자': 'date',
                                    '종목코드': 'code',
                                    '종목명': 'name',
                                    '거래량': 'volume',
                                    '시가': 'open',
                                    '고가': 'high',
                                    '저가': 'low',
                                    '종가': 'close'
                                })
    df = df.reset_index(drop=True)
    # datetime
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
    # code
    df.drop('name', axis=1, inplace=True)
    # target
    df['target'] = df.groupby('code')['close'].pct_change()
    df.fillna(0, inplace=True)   
    # rank
    df['rank'] = df.groupby('date')['target'].rank(method='first') - 1
    return df

df = get_target(final_df)
df.head()

,date,code,volume,open,high,low,close,target,rank
0,2021-06-01,A060310,166690,2890,2970,2885,2920,0.0,0.0
1,2021-06-01,A095570,63836,5860,5940,5750,5780,0.0,1.0
2,2021-06-01,A006840,103691,35500,35600,34150,34400,0.0,2.0
3,2021-06-01,A054620,462544,14600,14950,13800,14950,0.0,3.0
4,2021-06-01,A265520,131987,29150,29150,28800,29050,0.0,4.0


In [28]:
df

,date,code,volume,open,high,low,close,target,rank
0,2021-06-01,A060310,166690,2890,2970,2885,2920,0.000000,0.0
1,2021-06-01,A095570,63836,5860,5940,5750,5780,0.000000,1.0
2,2021-06-01,A006840,103691,35500,35600,34150,34400,0.000000,2.0
3,2021-06-01,A054620,462544,14600,14950,13800,14950,0.000000,3.0
4,2021-06-01,A265520,131987,29150,29150,28800,29050,0.000000,4.0
...,...,...,...,...,...,...,...,...,...
987995,2023-05-30,A189980,272284,3005,3035,2955,2980,-0.006667,475.0
987996,2023-05-30,A000540,50218,3250,3255,3195,3215,-0.006182,490.0
987997,2023-05-30,A003280,130664,1344,1395,1340,1370,0.017831,1580.0
987998,2023-05-30,A037440,141932,9170,9260,9170,9200,0.000000,903.0


In [29]:

sharpe_ratio = calc_spread_return_sharpe(df, portfolio_size=200, toprank_weight_ratio=2)
 
print(f'The Sharpe Ratio from June 1 to June 15 is {sharpe_ratio:.2f}')

The Sharpe Ratio from June 1 to June 15 is -5.60


,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...,...
987995,20230530,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,20230530,A000540,흥국화재,50218,3250,3255,3195,3215
987997,20230530,A003280,흥아해운,130664,1344,1395,1340,1370
987998,20230530,A037440,희림,141932,9170,9260,9170,9200


In [25]:
df_test = get_target(df_train)
df_test
sharpe_ratio = calc_spread_return_sharpe(df_test, portfolio_size=200, toprank_weight_ratio=2)
 
# print(f'The Sharpe Ratio from June 1 to June 15 is {sharpe_ratio:.2f}')

KeyError: 'rank'

In [15]:
sharpe_ratio = calc_spread_return_sharpe(df, portfolio_size=200, toprank_weight_ratio=2)

print(f'The Sharpe Ratio from June 1 to June 15 is {sharpe_ratio:.2f}')

The Sharpe Ratio from June 1 to June 15 is -167.51


In [11]:
import numpy as np
import pandas as pd

import pandas as pd
import numpy as np
from datetime import datetime

def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['rank'].min() == 0
        assert df['rank'].max() == len(df['rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='rank')['target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='rank', ascending=False)['target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio
# 가정한 데이터프레임 생성
data = {
    'date': pd.date_range(start='2023-06-01', end='2023-06-15').repeat(2000),  # 10개의 주식
    'stock': np.tile(['stock' + str(i) for i in range(2000)], 15),  # 15일
    'target': np.random.rand(2000*15),  # 예상 수익률을 임의로 생성
}

df = pd.DataFrame(data)
df['rank'] = df.groupby('date')['target'].rank(method='first') - 1  # 순위 생성

# calc_spread_return_sharpe 함수를 사용하여 6월 1일부터 6월 15일까지 Sharpe Ratio 계산
sharpe_ratio = calc_spread_return_sharpe(df, portfolio_size=20, toprank_weight_ratio=2)

print(f'The Sharpe Ratio from June 1 to June 15 is {sharpe_ratio:.2f}')

# calc_spread_return_sharpe 함수를 사용하여 6월 1일부터 6월 15일까지 Sharpe Ratio 계산
sharpe_ratio = calc_spread_return_sharpe(df, portfolio_size=100, toprank_weight_ratio=2)

print(f'The Sharpe Ratio from June 1 to June 15 is {sharpe_ratio:.2f}')

The Sharpe Ratio from June 1 to June 15 is -468.99
The Sharpe Ratio from June 1 to June 15 is -220.43


In [18]:
# '종목코드'와 '일자'로 정렬
df_train.sort_values(['종목코드', '일자'], inplace=True)
df_train

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
502,20210601,A000020,동화약품,114966,14700,14700,14450,14600
2502,20210602,A000020,동화약품,109559,14700,14700,14450,14500
4502,20210603,A000020,동화약품,96158,14550,14650,14450,14600
6502,20210604,A000020,동화약품,133900,14600,14800,14550,14700
8502,20210607,A000020,동화약품,511140,14800,15550,14750,15150
...,...,...,...,...,...,...,...,...
978145,20230523,A383800,LX홀딩스,150364,8390,8390,8310,8330
980145,20230524,A383800,LX홀딩스,122457,8310,8340,8280,8300
982145,20230525,A383800,LX홀딩스,84241,8300,8310,8270,8310
984145,20230526,A383800,LX홀딩스,126681,8300,8310,8270,8280


In [20]:
# '종목코드'별로 '종가'의 변화율 계산
df_train['target'] = df_train.groupby('종목코드')['종가'].pct_change()

print(df_train)

              일자     종목코드    종목명     거래량     시가     고가     저가     종가    target
502     20210601  A000020   동화약품  114966  14700  14700  14450  14600       NaN
2502    20210602  A000020   동화약품  109559  14700  14700  14450  14500 -0.006849
4502    20210603  A000020   동화약품   96158  14550  14650  14450  14600  0.006897
6502    20210604  A000020   동화약품  133900  14600  14800  14550  14700  0.006849
8502    20210607  A000020   동화약품  511140  14800  15550  14750  15150  0.030612
...          ...      ...    ...     ...    ...    ...    ...    ...       ...
978145  20230523  A383800  LX홀딩스  150364   8390   8390   8310   8330 -0.003589
980145  20230524  A383800  LX홀딩스  122457   8310   8340   8280   8300 -0.003601
982145  20230525  A383800  LX홀딩스   84241   8300   8310   8270   8310  0.001205
984145  20230526  A383800  LX홀딩스  126681   8300   8310   8270   8280 -0.003610
986145  20230530  A383800  LX홀딩스   70489   8300   8300   8270   8290  0.001208

[988000 rows x 9 columns]


In [22]:
df = df_train.rename(columns={
    '일자': 'date',
    '종목코드': 'code',
    '종목명': 'name',
    '거래량': 'volume',
    '시가': 'open',
    '고가': 'high',
    '저가': 'low',
    '종가': 'close'
})
df

,date,code,name,volume,open,high,low,close,target
502,20210601,A000020,동화약품,114966,14700,14700,14450,14600,NaN
2502,20210602,A000020,동화약품,109559,14700,14700,14450,14500,-0.006849
4502,20210603,A000020,동화약품,96158,14550,14650,14450,14600,0.006897
6502,20210604,A000020,동화약품,133900,14600,14800,14550,14700,0.006849
8502,20210607,A000020,동화약품,511140,14800,15550,14750,15150,0.030612
...,...,...,...,...,...,...,...,...,...
978145,20230523,A383800,LX홀딩스,150364,8390,8390,8310,8330,-0.003589
980145,20230524,A383800,LX홀딩스,122457,8310,8340,8280,8300,-0.003601
982145,20230525,A383800,LX홀딩스,84241,8300,8310,8270,8310,0.001205
984145,20230526,A383800,LX홀딩스,126681,8300,8310,8270,8280,-0.003610


In [21]:
df_train[df_train['종목명']=='삼성전자']

,일자,종목코드,종목명,거래량,시가,고가,저가,종가,target
746,20210601,A005930,삼성전자,14058401,80500,81300,80100,80600,NaN
2746,20210602,A005930,삼성전자,16414644,80400,81400,80300,80800,0.002481
4746,20210603,A005930,삼성전자,29546007,81300,83000,81100,82800,0.024752
6746,20210604,A005930,삼성전자,18112259,82700,82700,81500,82200,-0.007246
8746,20210607,A005930,삼성전자,16496197,82700,82800,81600,81900,-0.003650
...,...,...,...,...,...,...,...,...,...
978746,20230523,A005930,삼성전자,8561643,68500,68700,68100,68400,-0.001460
980746,20230524,A005930,삼성전자,8192896,68100,68700,68000,68500,0.001462
982746,20230525,A005930,삼성전자,14231160,69900,70000,68700,68800,0.004380
984746,20230526,A005930,삼성전자,19549511,69800,70400,69500,70300,0.021802
